In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import zscore

# Load your DataFrame first (df)
df = pd.read_csv('2425playerdata.csv')

# 1. Clean and map position
def map_position(pos):
    first = pos.split(',')[0].strip()
    if first.startswith(('RCB', 'LCB', 'CB', 'RB', 'LB', 'RWB', 'LWB')):
        return 'DEF'
    elif first.startswith(('LDMF', 'RDMF', 'DMF', 'LAMF', 'RAMF', 'AMF',
                           'CMF', 'RCMF', 'LCMF', 'MF', 'LW', 'RW')):
        return 'MF'
    elif first.startswith(('CF')):
        return 'ATT'
    else:
        return 'MF'

df['Position_updated'] = df['Position'].apply(map_position)
print(df)

# 2. Normalize within position groups
def normalize_within_position(df, func):
    num_cols = df.select_dtypes(include='number').columns
    return df.groupby('Position_updated', group_keys=False).apply(lambda g: func(g)).reset_index(drop=True)

# 3. Example usage
def sigmoid(x): return 100 / (1 + np.exp(-x))

df_normalized = normalize_within_position(df, lambda x: x.assign(**{col: sigmoid(zscore(x[col])) for col in x.select_dtypes(include='number').columns}))

In [ ]:
print(df_normalized)

In [ ]:
df_normalized.to_csv('clean2425playerdata.csv', index=False)